In [0]:
import numpy as np
import pylab as plt
#matplotlib inline
np.random.seed(17)

In [0]:
# set the integers
N = 2 ** 15
D = 2 ** 10
K = 2 ** 1

In [0]:
# make the fake data
ys = np.random.normal(size=(N, K))
trueA = np.vstack((np.ones(D), np.arange(-D/2 + 0.5, D/2, 1.)[None,:] / (D / 2.))).T
trueA_pseudo_inverse = np.linalg.solve(np.dot(trueA.T, trueA), trueA.T)
xs = np.dot(trueA, ys.T).T
xs += 2. ** -2 * np.random.normal(size=(N, D))

In [0]:
for n in [17, 42, 4097, 17001]:
  plt.plot(xs[n])
  plt.text(D, xs[n, -1], "({:0.2f}, {:0.2f})".format(ys[n, 0], ys[n, 1]))

In [0]:
# make a training set
N_train = 2 ** 7 + 5
assert 2 * N_train < N
xs_train = xs[:N_train, :]
ys_train = ys[:N_train, :]

In [0]:
# make test set
N_test = 2 ** 8
assert N_train + N_test < N
xs_test = xs[-N_test:, :]
ys_test = ys[-N_test:, :]

In [0]:
# train generative model: x = A y + noise
A = np.zeros((D, K))
for d in range(D):
  A[d, :] = np.linalg.solve(np.dot(ys_train.T, ys_train), np.dot(ys_train.T, xs_train[:, d]))
A_pseudo_inverse = np.linalg.solve(np.dot(A.T, A), A.T)

In [0]:
# train discriminative model: y = B x + noise -- now with L2 regularization!
lam = 2 ** -30
B = np.zeros((K, D))
for k in range(K):
  B[k, :] = np.linalg.solve(np.dot(xs_train.T, xs_train) + lam * np.eye(D), np.dot(xs_train.T, ys_train[:, k]))

In [0]:
# test the models
ys_generative_test = np.dot(A_pseudo_inverse, xs_test.T).T
ys_discriminative_test = np.dot(B, xs_test.T).T

In [0]:
plt.scatter(ys_test[:, 1], ys_generative_test[:, 1])
plt.title("test of the generative model")
plt.xlabel("true slope")
plt.ylabel("inferred slope")
plt.plot([-3, 3], [-3, 3], "k-", alpha=0.25)

In [0]:
plt.scatter(ys_test[:, 1], ys_discriminative_test[:, 1])
plt.title("test of the discriminative model")
plt.xlabel("true slope")
plt.ylabel("estimated slope")
plt.plot([-3, 3], [-3, 3], "k-", alpha=0.25)

In [0]:
for k in range(K):
  plt.plot(trueA_pseudo_inverse[k, :])
plt.title("true (information-theory-optimal) result")

In [0]:
for k in range(K):
  plt.plot(A_pseudo_inverse[k, :])
plt.title("pseudo-inverse of generative-model A")

In [0]:
for k in range(K):
  plt.plot(B[k, :])
plt.title("discriminative-model B")